# Generate Graph Database
*AJH 07/08/20 for S2DS Ditchley Foundation project*

This notebook demonstrates how to generate and to visualise a Neo4j graph database from .csv files using python packages.

In [1]:
# import the necessary python packages
from py2neo import Graph
from py2neo.data import Node, Relationship

In [5]:
# load / declare the database
graph = Graph("bolt://localhost:7687", user="bluetit", password="tweetoftheday")
graph.begin()

# start with an empty graph
graph.delete_all()

The mock twitter user information is stored in a csv file. To load it into the graph we wrap a cypher query into a string and then run that query.

In [6]:
# load in twitter user information
query_string = '''
   LOAD CSV WITH HEADERS FROM "file:///twitter_dummy_users.csv" AS row
   CREATE (n:Person)
   SET n = row;
   '''
# run cypher query
graph.run(query_string)

The follower information is also stored in a csv file. This file contains a list of pairs of ID numbers for the follower and the user being followed.

In [7]:
# load in file containing follower information
query_string = '''
   LOAD CSV WITH HEADERS FROM 'file:///twitter_dummy_followers.csv' AS row
   MATCH (p:Person) WHERE p.userID = row.userID
   MATCH (n:Person) WHERE n.userID = row.followerID
   CREATE (n)-[:Follows]->(p)
   '''
# runcypher query
graph.run(query_string)

Let's check that the graph has been drawn by counting the number of nodes and edges.

In [8]:
#check nodes and edges have been created
print('No. nodes = ',len(graph.nodes))
print('No. edges = ',len(graph.relationships))

No. nodes =  6
No. edges =  13


We would like to visualise our graph. Let's use the jgraph package to make a 3D visualisation of the edges and nodes.

In [10]:
import jgraph

# run a query returning the 'Follows' edges 
data = graph.run("MATCH (a:Person)-[r:Follows]->(b:Person) RETURN a.userID, b.userID")

# initialise a list
list_of_followers = []

# put the follower information into a list of tuples to be read by jgraph
for d in data:
    list_of_followers.append((tuple([int(d[0]),int(d[1])])))

# draw the graph
jgraph.draw(list_of_followers)